In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
#import util
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time
import os

from scipy.optimize import fmin_powell
from scipy import integrate

import theano as thno
import theano.tensor as T
import arviz as az

az.style.use('arviz-darkgrid')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AdditionalAttributes(BaseEstimator, TransformerMixin):
    def __init__(self, include_additional_attr = True): # no *args or **kargs
        self.include_additional_attr = include_additional_attr
        self.guille_feature_names = ['H', 'dest_A', 'dest_I', 'dest_dTR', 
                                     'dest_hK', 'dest_hM', 'dest_mR', 'src_A', 
                                     'src_I', 'src_dTR', 'src_hK', 'src_hM','src_mR']
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        if self.include_additional_attr:
            return X # might need to check if this returns pandas or a numpy, i need a numpy
        else:
            return X[self.guille_feature_names]

In [ ]:
from joblib import dump

def save_models(saveas, rnd_estimator, lr_estimator):
    save_to = '../models/'
    saveas = os.path.join(save_to, saveas)
    if os.path.exists(saveas):
        raise FileExistsError(f'The path "{os.path.abspath(saveas)}" already exists, ' 
                              'please choose a new name.')
    else:
        rfc_dir = os.path.join(saveas, 'rfc')
        bayes_dir = os.path.join(saveas, 'bayes/')
        
        os.makedirs(rfc_dir)
        os.mkdir(bayes_dir)
    
    # save random forest estimator
    dump(rnd_estimator, os.path.join(rfc_dir, 'model.rnd'))
    
    # save bayesian logistic regression model
    lr_estimator.save(bayes_dir)

Set Project Parameters

In [ ]:
include_additional_attr = False
dataset_filepath = "../../big-network-attr.csv"
dataset_key = 'nZqfQwwiTxtRmiJ'

# Explore Dataset

In [ ]:
# dataset = pd.read_csv(dataset_filepath)
dataset = pd.read_hdf(dataset_filepath, key=dataset_key)

# Remove user_ids from dataframe
dataset = dataset.drop(['src_user_id', 'dest_user_id'], axis=1)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
sns.stripplot(x="y", y="H", data=dataset, jitter=True)

In [ ]:
X = dataset.drop("y", axis=1) # drop labels for training set
y = dataset["y"].copy()

feature_names = X.columns.values

y.value_counts()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline = Pipeline([
    ('attribs_adder', AdditionalAttributes(include_additional_attr = include_additional_attr)),
    ('imputer', SimpleImputer(strategy="mean")),
])

# ('std_scaler', StandardScaler()),

X = pipeline.fit_transform(X)

In [ ]:
# resample using SMOTE balance
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X, y)

In [ ]:
# column values based on whether additional attributes is true or false
# include_additional_attr = pipeline.named_steps['attribs_adder'].include_additional_attr
guille_feature_names = pipeline.named_steps['attribs_adder'].guille_feature_names

if not include_additional_attr:
    feature_names = guille_feature_names

In [ ]:
X_resampled = pd.DataFrame(X_resampled, columns=feature_names)
y_resampled = pd.Series(y_resampled)

In [ ]:
y_resampled.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
print ('x_train shape: ', X_train.shape)
print ('y_train shape: ', y_train.shape)
print ('x_test shape: ', X_test.shape)
print ('y_test shape: ', y_test.shape)

## Exploring the data

In [ ]:
#g = seaborn.pairplot(data)

In [ ]:
# Compute the correlation matrix
corr = X_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)



# Get most important features using Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(random_state=42)

params = {
    'max_features': [.4,'auto', 4], 
    'n_estimators': [15, 100, 200], 
    'min_samples_leaf': [.1, 1],
}

cv = GridSearchCV(rfc, params, n_jobs=-1).fit(X_train, y_train)

clf = cv.best_estimator_
cv.best_params_

In [ ]:
from sklearn.model_selection import cross_val_score

# Conduct cross-validation
acc_cv_results = cross_val_score(clf,
                             X_train, # Feature matrix
                             y_train, # Target vector
                             cv=10, # Cross-validation technique
                             scoring="accuracy",
                             n_jobs=-1) # Use all CPU scores

f1_cv_results = cross_val_score(clf,
                             X_train, # Feature matrix
                             y_train, # Target vector
                             cv=10, # Cross-validation technique
                             scoring="f1",
                             n_jobs=-1) # Use all CPU scores

print(f'accuracy: {acc_cv_results.mean()}\nf1_score: {f1_cv_results.mean()}')

In [ ]:
classes = [0, 1]

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
#mapping = {1:'negative', 0:'positive'}
fig, ax = plt.subplots(figsize=(7, 7))
cm_viz = ConfusionMatrix(clf, classes=classes, fontsize=15)
cm_viz.fit(X_train, y_train)
cm_viz.score(X_test, y_test)
cm_viz.poof()

In [ ]:
from yellowbrick.classifier import ClassificationReport
fig, ax = plt.subplots(figsize=(10, 7))
viz = ClassificationReport(clf, classes=classes, fontsize=15)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.poof()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
importances = clf.feature_importances_

feat_importances = pd.Series(importances, index=feature_names)
n_features = feat_importances.nlargest(15)
n_features

In [ ]:
features = n_features.index

## Build Bayesian Logistics Regression Model

In [ ]:
X = X_resampled[features]
y = y_resampled.copy()

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print ('x_train shape: ', X_train.shape)
print ('y_train shape: ', y_train.shape)
print ('x_test shape: ', X_test.shape)
print ('y_test shape: ', y_test.shape)

In [ ]:
#g = seaborn.pairplot(data)

In [ ]:
# Compute the correlation matrix
corr = X_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)



In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
from pymc3_models.models.LogisticRegression import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train, y_train, inference_type='nuts', inference_args={'draws': 2000})

In [ ]:
az.plot_trace(LR.trace)

In [ ]:
LR.summary

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
y_score = LR.predict_proba(X_test)

### Performance

In [ ]:
LR.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['1', '0']);

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_score)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_test, y_score))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
save_models(saveas='big-network-full-attr', rnd_estimator=clf, lr_estimator=LR)